In [ ]:
import sys
print(sys.getdefaultencoding())

In [ ]:
import csv
import os

DATADIR = "DataWranglingwithMongoDB/"
DATAFILE = "beatles-diskography.csv"
DATAFILE = os.path.join(DATADIR, DATAFILE)

def parse_file(datafile):
    data = []
    num = 0
    with open(datafile, encoding='utf-8', mode='r') as f:
        for line in f:
            line = line.strip('\n') 
            if num == 0:
                Keys = line.split(',')
            if num > 0 and num < 11:
                Values = line.split(',')
                beatles = dict(zip(Keys, Values))
                data.append(beatles) 
            num += 1
    return data

In [ ]:
data = parse_file(DATAFILE)

In [ ]:
print(data)

In [ ]:
"""
Your task is as follows:
- read the provided Excel file
- find and return the min, max and average values for the COAST region
- find and return the time value for the min and max entries
- the time values should be returned as Python tuples

Please see the test function for the expected return format
"""

import xlrd
import os
import numpy as np

from zipfile import ZipFile
datafile = "2013_ERCOT_Hourly_Load_Data.xls"
datadir = "./DataWranglingwithMongoDB/"

datafile = os.path.join(datadir,datafile)

def open_zip(datafile):
    with ZipFile('{0}.zip'.format(datafile.rsplit('.',1)[0]), 'r') as myzip:
        myzip.extractall(datadir)
       
def parse_file(datafile):
    workbook = xlrd.open_workbook(datafile)
    sheet = workbook.sheet_by_index(0)

    ### example on how you can get the data
    #sheet_data = [[sheet.cell_value(r, col) for col in range(sheet.ncols)] for r in range(sheet.nrows)]

    ### other useful methods:
    print("\nROWS, COLUMNS, and CELLS:")
    print("Number of rows in the sheet:")
    print(sheet.nrows)
    print("Type of data in cell (row 3, col 2):") 
    print(sheet.cell_type(3, 2))
    print ("Value in cell (row 0, col 4):", )
    
    zones = 'COAST, EAST, FAR_WEST, NORTH, NORTH_C, SOUTHERN, SOUTH_C, WEST'
    zones = zones.split(sep=',')
    zones = [zone.strip() for zone in zones]
    
    print(zones)
    for zone in zones:
        for x in range(0,10):
            if sheet.cell_value(0, x) == zone:
                column = np.array(sheet.col_values(x, start_rowx=1))
                maxidx = column.argmax()
                maxdate_raw = xlrd.xldate_as_tuple(sheet.cell_value(maxidx + 1, 0),0)
                maxdate = list(maxdate_raw[0:4])
                
                print(maxdate)
                max = column[maxidx]
                
    print("Get a slice of values in column 2 - COAST from rows 1-3:")

    coast_vals = sheet.col_values(1, start_rowx=1)
    
    print
    print("\nDATES:")
    print("Type of data in cell (row 1, col 0):")
    print(sheet.cell_type(1, 0))
    exceltime = sheet.cell_value(1, 0)
    print("Time in Excel format:",)
    print(exceltime)
    print("Convert time to a Python datetime tuple, from the Excel float:")
    print(xlrd.xldate_as_tuple(exceltime, 0))

    coast_vals = np.array(coast_vals)
    maxidx = coast_vals.argmax()
    minidx = coast_vals.argmin()
    maxtime = xlrd.xldate_as_tuple(sheet.cell_value(maxidx + 1, 0),0)
    mintime = xlrd.xldate_as_tuple(sheet.cell_value(minidx + 1, 0),0)
    
    data = {
            'maxtime': maxtime,
            'maxvalue': np.max(coast_vals),
            'mintime': mintime,
            'minvalue': np.min(coast_vals),
            'avgcoast': np.average(coast_vals)
    }
    
    print(data)
    return data

open_zip(datafile)
data = parse_file(datafile)
#print('{0}.zip'.format(datafile))

In [ ]:
"""
Your task is as follows:
- read the provided Excel file
- find and return the min, max and average values for the COAST region
- find and return the time value for the min and max entries
- the time values should be returned as Python tuples

Please see the test function for the expected return format
"""

import xlrd
import os
import csv
import numpy as np

from zipfile import ZipFile
datafile = "2013_ERCOT_Hourly_Load_Data.xls"
datadir = "./DataWranglingwithMongoDB/"
datafile = os.path.join(datadir,datafile)

def open_zip(datafile):
    with ZipFile('{0}.zip'.format(datafile.rsplit('.',1)[0]), 'r') as myzip:
        myzip.extractall(datadir)
       
def parse_file(datafile):
    workbook = xlrd.open_workbook(datafile)
    sheet = workbook.sheet_by_index(0)
    
    zones = 'Station,COAST, EAST, FAR_WEST, NORTH, NORTH_C, SOUTHERN, SOUTH_C, WEST'
    zones = zones.split(sep=',')
    zones = [zone.strip() for zone in zones]
    
    data = []
    
    print(zones)
    for zone in zones:
        for x in range(0,10):
            if sheet.cell_value(0, x) == zone:
                column = np.array(sheet.col_values(x, start_rowx=1))
                maxidx = column.argmax()
                maxdate = xlrd.xldate_as_tuple(sheet.cell_value(maxidx + 1, 0),0)
                line = list(maxdate[0:4])
                line.append(column[maxidx])
                line.insert(0,zone)
                data.append(line)
                
    return data

def save_file(data, filename):
    with open(filename, 'w') as csvfile:
        
        writer = csv.writer(csvfile, delimiter='|')
        
        fieldnames = ['Station', 'Year', 'Month', 'Day', 'Hour', 'Max Load']
        writer.writerow(fieldnames)
        
        #writer = csv.writer(csvfile, delimiter='|')
        writer.writerows(data)
    
open_zip(datafile)

filename = "2013_Max_Loads.csv"
data = parse_file(datafile)
save_file(data, filename)


#print('{0}.zip'.format(datafile))

In [ ]:
str = 'Station|Year|Month|Day|Hour|Max Load'
str.split(sep = '|')

In [ ]:
def test():
    open_zip(datafile)
    data = parse_file(datafile)

    assert data['maxtime'] == (2013, 8, 13, 17, 0, 0)
    assert round(data['maxvalue'], 10) == round(18779.02551, 10)


test()

In [ ]:
# To experiment with this code freely you will have to run this code locally.
# Take a look at the main() function for an example of how to use the code.
# We have provided example json output in the other code editor tabs for you to
# look at, but you will not be able to run any queries through our UI.
import json
import requests


BASE_URL = "http://musicbrainz.org/ws/2/"
ARTIST_URL = BASE_URL + "artist/"

# query parameters are given to the requests.get function as a dictionary; this
# variable contains some starter parameters.
query_type = {  "simple": {},
                "atr": {"inc": "aliases+tags+ratings"},
                "aliases": {"inc": "aliases"},
                "releases": {"inc": "releases"}}


def query_site(url, params, uid="", fmt="json"):
    # This is the main function for making queries to the musicbrainz API.
    # A json document should be returned by the query.
    params["fmt"] = fmt
    r = requests.get(url + uid, params=params)
    print("requesting", r.url)
    if r.status_code == requests.codes.ok:
        return r.json()
    else:
        r.raise_for_status()


def query_by_name(url, params, name):
    # This adds an artist name to the query parameters before making
    # an API call to the function above.
    params["query"] = "artist:" + name
    return query_site(url, params)


def pretty_print(data, indent=4):
    # After we get our output, we can format it to be more readable
    # by using this function.
    if type(data) == dict:
        print(json.dumps(data, indent=indent, sort_keys=True))
    else:
        print(data)


def main():
    '''
    Modify the function calls and indexing below to answer the questions on
    the next quiz. HINT: Note how the output we get from the site is a
    multi-level JSON document, so try making print statements to step through
    the structure one level at a time or copy the output to a separate output
    file.
    '''
    results = query_by_name(ARTIST_URL, query_type["simple"], "Queen")
    pretty_print(results)

    artist_id = results["artists"][1]["id"]
    print ("\nARTIST:")
    pretty_print(results["artists"][1])

    artist_data = query_site(ARTIST_URL, query_type["releases"], artist_id)
    releases = artist_data["releases"]
    print("\nONE RELEASE:")
    pretty_print(releases[0], indent=2)
    release_titles = [r["title"] for r in releases]

    print("\nALL TITLES:")
    for t in release_titles:
        print (t)


#if __name__ == '__main__':
#    main()

In [ ]:
results = query_by_name(ARTIST_URL, query_type["simple"], "One Direction")
print([ artist["name"] for artist in results["artists"] ])
#pretty_print(results)


In [ ]:
results["artists"][0]

In [ ]:
#!/usr/bin/env python
"""
Your task is to process the supplied file and use the csv module to extract data from it.
The data comes from NREL (National Renewable Energy Laboratory) website. Each file
contains information from one meteorological station, in particular - about amount of
solar and wind energy for each hour of day.

Note that the first line of the datafile is neither data entry, nor header. It is a line
describing the data source. You should extract the name of the station from it.

The data should be returned as a list of lists (not dictionaries).
You can use the csv modules "reader" method to get data in such format.
Another useful method is next() - to get the next line from the iterator.
You should only change the parse_file function.
"""
import csv
import os

DATADIR = "./data/"
DATAFILE = "745090.csv"

def parse_file(datafile):
    name = ""
    data = []
    with open(datafile,mode  = 'rt') as f:
        reader = csv.reader(f)
        name = next(reader)[1]
        column_names = next(reader) # keep column names separated
        
        #for row in reader:
        #    data.append(row)
        # cuter way of achieving this is a list comprehension used by the 
        # instructor:
        data = [row for row in reader]
    # Do not change the line below
    return (name, data)

def test():
    datafile = os.path.join(DATADIR, DATAFILE)
    name, data = parse_file(datafile)

    assert name == "MOUNTAIN VIEW MOFFETT FLD NAS"
    assert data[0][1] == "01:00"
    assert data[2][0] == "01/01/2005"
    assert data[2][5] == "2"


if __name__ == "__main__":
    test()

In [ ]:
datafile = os.path.join(DATADIR,DATAFILE)
parsed_data = parse_file(datafile)

name = parsed_data[0]
data = parsed_data[1]

print(data[0][1])

In [ ]:
print(name)

In [46]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
This exercise shows some important concepts that you should be aware about:
- using codecs module to write unicode files
- using authentication with web APIs
- using offset when accessing web APIs

To run this code locally you have to register at the NYTimes developer site 
and get your own API key. You will be able to complete this exercise in our UI without doing so,
as we have provided a sample result.

Your task is to process the saved file that represents the most popular (by view count)
articles in the last day, and return the following data:
- list of dictionaries, where the dictionary key is "section" and value is "title"
- list of URLs for all media entries with "format": "Standard Thumbnail"

All your changes should be in the article_overview function.
The rest of functions are provided for your convenience, if you want to access the API by yourself.
"""
import json
import codecs
import requests

URL_MAIN = "http://api.nytimes.com/svc/"
URL_POPULAR = URL_MAIN + "mostpopular/v2/"
API_KEY = { "popular": "",
            "article": ""}


def get_from_file(kind, period):
    filename = "popular-{0}-{1}.json".format(kind, period)
    with open(filename, "r") as f:
        return json.loads(f.read())


def article_overview_old(kind, period):
    data = get_from_file(kind, period)
    titles = []
    urls =[]
    
    [titles.append(x['title']) for x in data]
    for x in data:
        if isinstance( x['media'] , list):
            journal = x['media'][0]
            for key in journal:
                if isinstance(journal[key], list):
                    media = journal[key][0]
                    if media['format'] == 'Standard Thumbnail': 
                        print(media['url'])
        if isinstance( x['media'] , dict):
            print(type(x['media']['media-metadata']))
    return (titles, urls)

def article_overview(kind, period):
    data = get_from_file(kind, period)
    titles = []
    urls =[]
    
    [titles.append({x['section'] : x['title']}) for x in data]
    
    for journal in data:
        for media in journal['media']:
            for key in media:
                if key == "media-metadata":
                    for graphic in media[key]:
                        for key in graphic:
                            if key == "format":
                                if graphic[key] == "Standard Thumbnail":
                                    urls.append(graphic['url'])
                
    return (titles, urls)

def query_site(url, target, offset):
    # This will set up the query with the API key and offset
    # Web services often use offset paramter to return data in small chunks
    # NYTimes returns 20 articles per request, if you want the next 20
    # You have to provide the offset parameter
    if API_KEY["popular"] == "" or API_KEY["article"] == "":
        print("You need to register for NYTimes Developer account to run this program.")
        print("See Intructor notes for information")
        return False
    params = {"api-key": API_KEY[target], "offset": offset}
    r = requests.get(url, params = params)

    if r.status_code == requests.codes.ok:
        return r.json()
    else:
        r.raise_for_status()


def get_popular(url, kind, days, section="all-sections", offset=0):
    # This function will construct the query according to the requirements of the site
    # and return the data, or print an error message if called incorrectly
    if days not in [1,7,30]:
        print("Time period can be 1,7, 30 days only")
        return False
    if kind not in ["viewed", "shared", "emailed"]:
        print("kind can be only one of viewed/shared/emailed")
        return False

    url += "most{0}/{1}/{2}.json".format(kind, section, days)
    data = query_site(url, "popular", offset)

    return data


def save_file(kind, period):
    # This will process all results, by calling the API repeatedly with supplied offset value,
    # combine the data and then write all results in a file.
    data = get_popular(URL_POPULAR, "viewed", 1)
    num_results = data["num_results"]
    full_data = []
    with codecs.open("popular-{0}-{1}.json".format(kind, period), encoding='utf-8', mode='w') as v:
        for offset in range(0, num_results, 20):        
            data = get_popular(URL_POPULAR, kind, period, offset=offset)
            full_data += data["results"]
        
        v.write(json.dumps(full_data, indent=2))

"""
def test():
    titles, urls = article_overview("viewed", 1)
    assert len(titles) == 20
    assert len(urls) == 30
    assert titles[2] == {'Opinion': 'Professors, We Need You!'}
    assert urls[20] == 'http://graphics8.nytimes.com/images/2014/02/17/sports/ICEDANCE/ICEDANCE-thumbStandard.jpg'


if __name__ == "__main__":
    test()
"""

'\ndef test():\n    titles, urls = article_overview("viewed", 1)\n    assert len(titles) == 20\n    assert len(urls) == 30\n    assert titles[2] == {\'Opinion\': \'Professors, We Need You!\'}\n    assert urls[20] == \'http://graphics8.nytimes.com/images/2014/02/17/sports/ICEDANCE/ICEDANCE-thumbStandard.jpg\'\n\n\nif __name__ == "__main__":\n    test()\n'

In [47]:
kind = 'viewed'
period = 1
stuff = article_overview(kind,period)
for i in stuff[0]:
    print(i)
for i in stuff[1]:
    print(i)
print(len(stuff[1]))

{'Opinion': 'The All-or-Nothing Marriage'}
{'Opinion': 'Marry First, Then Cheat'}
{'Opinion': 'Professors, We Need You!'}
{'N.Y. / Region': 'Common Core Curriculum Now Has Critics on the Left'}
{'Fashion & Style': 'For Interns, All Work and No Payoff'}
{'Sports': 'NBC Pushes Too Far in Bringing Bode Miller to Tears'}
{'Opinion': 'Barons of Broadband'}
{'Sports': 'A Mad Dash for Salt Rescues Olympic Slopes'}
{'U.S.': 'Spying by N.S.A. Ally Entangled U.S. Law Firm'}
{'N.Y. / Region': 'Addicted on Staten Island'}
{'Business Day': 'Saving an Endangered British Species: The Pub'}
{'Sports': 'Behind Meryl Davis and Charlie White, U.S. Is Close to Its First Ice Dance Gold'}
{'Technology': 'Intel’s Sharp-Eyed Social Scientist'}
{'Sports': 'Hanyu Falls Twice, but Still Wins Gold'}
{'Opinion': 'Is the Universe a Simulation?'}
{'Opinion': 'Parental Pity Party'}
{'Opinion': 'Behind Bars'}
{'Opinion': 'A Watchful Eye in Hospitals'}
{'Movies': 'The Greatest Film Role Ever Cast'}
{'Business Day': 'Co